In [1]:
import os, math
import pandas as pd
import numpy as np

In [2]:
from gensim.models import Word2Vec as w2v
from os import path as osp

root = osp.dirname(os.getcwd())
model = w2v.load(osp.join(root,'model/stackexchange_model'))

In [3]:
def get_closest(word, n):
    '''Get n most similar words by words.'''
    #This function can easily be expanded to get similar words to phrases--
    #using sent2vec() method defined in WithWord2Vec notebook. 
    word = word.lower()
    words = [word]
    similar_vals = [1]
    try:
        similar_list = model.most_similar(positive=[word],topn=n)
        
        for tupl in similar_list:
            words.append(tupl[0])
            similar_vals.append(tupl[1])
    except:
        #If word not in vocabulary return same word and 1 similarity-- 
        #see initialisation of words, similarities.
        pass
    
    return words, similar_vals

In [6]:
cvs = pd.read_csv('prc_data.csv', sep='\t')
cvs = cvs.set_index('Unnamed: 0')

prc_description = "software engineer c c java uml xamp agile defencecommunication client successful expand company develop air defence system information system target system communication system require additional software engineer support design evolution requirement software code identification test requirement development component test harness support design code review activity derisk design evolution qualification software engineer degree software engineering computer science math physics equivalent good degree pref 1st 1 meng equiv essential requirement software engineer experience follow object orient analysis design preferably uml unix linux window c c java html php candidate defence background experience agile development advantage experience xamp technology useful desirable requirement software engineer experience follow realtime design mathematical algorithm cod network protocol oo case tool structure design method datum voice network computer telephony integration cti appreciation follow test technique strategy configuration management personal attribute flexibility adaptability team player good communication skill discipline company provide excellent career prospect career development recent software graduate consider role keyword software engineer c c java realtime embed uml linux unix oo agile xamp defence communication system "

In [8]:
word_value = {}
similar_words_needed = 2
for word in prc_description.split():
    similar_words, similarity = get_closest(word, similar_words_needed)
    for i in range(len(similar_words)):
        word_value[similar_words[i]] = word_value.get(similar_words[i], 0)+similarity[i]
        print(similar_words[i], word_value[similar_words[i]])

/Users/prateekgupta/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:9: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


software 1
sofware 0.623483419418335
softwares 0.5280500650405884
engineer 1
engineering 0.6810150146484375
architect 0.6114804744720459
c 1
seudoc 0.5755490064620972
pseudoc 0.5684827566146851
c 2
seudoc 1.1510980129241943
pseudoc 1.1369655132293701
java 1
scala 0.650192379951477
jdk 0.6426096558570862
uml 1
diagrams 0.6789827346801758
eer 0.6273860931396484
xamp 1
wampserver 0.5981194376945496
xammp 0.5898818373680115
agile 1
scrum 0.7460435032844543
devops 0.6338268518447876
defencecommunication 1
client 1
clients 0.6456912755966187
server 0.5712165832519531
successful 1
unsuccessful 0.6726938486099243
succesful 0.6204940676689148
expand 1
extend 0.6933382153511047
expanding 0.6305662989616394
company 1
organization 0.691008448600769
department 0.6829237937927246
develop 1
developing 0.6968998908996582
implement 0.6211676597595215
air 1
pro 0.59244704246521
water 0.5711874961853027
defence 1
defense 0.752550482749939
defending 0.49757060408592224
system 1
systems 0.6141223311424255


In [10]:
no_of_cv = 150

count = {}
idf = {}
for word in word_value.keys():
    count[word] = 0
    for i in range(no_of_cv):
        try:
            if word in cvs.loc(0)['skill'][i].split() or word in cvs.loc(0)['exp'][i].split():
                count[word] += 1
        except:
            pass
    if (count[word] == 0):
        count[word] = 1
    idf[word] = math.log(no_of_cv/count[word])
print(count)
print(idf)

{'consent': 1, 'test': 31, 'kmail': 1, 'industry': 18, 'keywords': 1, 'defencecommunication': 1, 'testing': 38, 'calibrate': 1, 'sytem': 1, 'xamp': 1, 'equal': 1, 'identification': 2, 'html': 83, 'uml': 7, 'chemistry': 1, 'sofware': 1, 'background': 8, 'supports': 1, 'arrange': 1, 'mathematics': 1, 'analyses': 1, 'engineers': 1, 'leeway': 1, 'team': 64, 'qualification': 2, 'personal': 1, 'strategies': 1, 'physics': 1, 'philosophy': 1, 'cod': 10, 'unsuccessful': 1, 'designers': 1, 'degrees': 1, 'inject': 1, 'skills': 45, 'defense': 1, 'thunderbird': 1, 'require': 18, 'company': 37, 'agile': 21, 'systems': 14, 'target': 8, 'staff': 1, 'restriction': 1, 'method': 9, 'education': 3, 'tests': 1, 'additional': 1, 'grad': 1, 'pro': 2, 'bmod': 1, 'destination': 4, 'heliostats': 1, 'harnesses': 1, 'player': 18, 'clients': 1, 'wampserver': 1, 'mgmt': 1, 'configuration': 10, 'biometric': 1, 'career': 12, 'beneficial': 1, 'disadvantage': 1, 'xammp': 1, 'followed': 4, 'bad': 1, 'defence': 1, 'analy

In [11]:
score = {}
for i in range(no_of_cv):
    score[i] = 0
    try:
        for word in word_value.keys():
            tf = cvs.loc(0)['skill'][i].split().count(word) + cvs.loc(0)['exp'][i].split().count(word)
            score[i] += word_value[word]*tf*idf[word]
    except:
        pass

In [13]:
sorted_list = []
for i in range(no_of_cv):
    sorted_list.append((score[i], i))
    
sorted_list.sort(reverse = True)

for s, i in sorted_list:
    if list(cvs)[i] != '.DS_Store':
        print(list(cvs)[i], ':', s)

cv135 : 944.4421166158238
cv122 : 944.4421166158238
cv109 : 944.4421166158238
cv195 : 907.3636292235202
cv215 : 832.660861836115
cv204 : 818.031645646612
cv207 : 758.4341693748713
cv205 : 596.6990768676358
cv174 : 578.9138648330645
cv217 : 574.6691408070324
cv144 : 490.6658618446015
cv191 : 471.76701556751533
cv199 : 462.0207678755738
cv142 : 355.3811329793244
cv123 : 326.44972487501906
cv113 : 307.0156691865352
cv107 : 301.666342384623
cv181 : 288.26289637530584
cv124 : 282.60151512754334
cv136 : 268.9558126342039
cv102 : 268.67200217076095
cv139 : 258.5049454605583
cv112 : 252.4072329652164
cv154 : 193.97353428320002
cv119 : 178.84376670564507
cv187 : 165.98465241048564
cv130 : 165.3125014899856
cv22 : 163.70870940263262
cv19 : 161.55555829955807
cv140 : 152.2014863250215
cv111 : 152.06927329086867
cv213 : 144.46986000982437
cv153 : 141.15936962427236
cv13 : 138.8386487525239
cv197 : 121.6161781111421
cv224 : 120.77335585640296
cv104 : 120.14207460484393
cv116 : 117.19568245954602
cv